In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "NF-UNSW-NB15-v2.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2151027
1      91904
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
0    2295222
1      95053
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,123,123,123,123,123,123,123,123,123,123,...,123,123,123,123,123,123,123,123,123,123
Backdoor,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
Benign,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103,...,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103
DoS,505,505,505,505,505,505,505,505,505,505,...,505,505,505,505,505,505,505,505,505,505
Exploits,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882,...,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882
Fuzzers,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559,...,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559
Generic,476,476,476,476,476,476,476,476,476,476,...,476,476,476,476,476,476,476,476,476,476
Reconnaissance,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
Shellcode,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159


In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
Backdoor,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
Benign,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522,...,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522
DoS,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
Exploits,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155,...,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155
Fuzzers,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231,...,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231
Generic,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656,...,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656
Reconnaissance,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,...,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Shellcode,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143


In [16]:
# X = data.drop(columns=["Attack", "Label"])
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X_train, y_train = X, y
X_test = test_data.drop(columns=["Attack", "Label"])
y_test = test_data[["Attack", "Label"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape
##

((224295, 41), (239027, 41), (224295, 2), (239027, 2))

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
28228,175.45.176.3,149.171.126.19,0.000002,0.000004,0.028485,0.000341,0.045794,0.000341,5.420943e-07,5.797048e-07,...,0.0,0.558214,0.558214,0.000007,0.000007,0.000002,0.000002,0.0,0.000002,"[1.7197633617740213e-06, 4.298851310470571e-06..."
856405,175.45.176.1,149.171.126.11,0.000001,0.000003,0.022255,0.000273,0.035816,0.000219,4.349885e-07,4.651680e-07,...,0.0,0.447924,0.447924,0.000009,0.000009,0.000001,0.000001,0.0,0.000001,"[1.3799764291286633e-06, 3.449494048215318e-06..."
465045,175.45.176.1,149.171.126.11,0.000002,0.000005,0.031897,0.000394,0.043554,0.000315,6.265204e-07,6.699884e-07,...,0.0,0.645151,0.645151,0.000005,0.000005,0.000002,0.000002,0.0,0.000002,"[1.987600574078275e-06, 4.968357579006963e-06,..."
959848,175.45.176.3,149.171.126.10,0.000002,0.000005,0.029782,0.000370,0.047666,0.000370,5.878823e-07,6.286695e-07,...,0.0,0.605364,0.605364,0.000013,0.000013,0.000002,0.000002,0.0,0.000002,"[1.8650231844538396e-06, 4.661953812225061e-06..."
1381572,175.45.176.0,149.171.126.14,0.000002,0.000004,0.028129,0.000351,0.038160,0.000281,5.580212e-07,5.967367e-07,...,0.0,0.574615,0.574615,0.000016,0.000016,0.000002,0.000002,0.0,0.000002,"[1.7702906951465035e-06, 4.425153276258943e-06..."


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph
train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_UNSW_v3.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [32]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v3.pkl'))

<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [38]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.059827,-0.062374,0.058713,0.079709,0.041607,0.097148,0.185068,0.175076,0.037400,-0.011215,...,0.007576,0.006639,-0.105099,0.082139,0.258052,0.175949,0.073437,0.155727,Analysis,1
1,0.059827,-0.062374,0.058713,0.079709,0.041607,0.097148,0.185068,0.175076,0.037400,-0.011215,...,0.007576,0.006639,-0.105099,0.082139,0.258052,0.175949,0.073437,0.155727,Analysis,1
2,0.059827,-0.062374,0.058713,0.079709,0.041607,0.097148,0.185068,0.175076,0.037400,-0.011215,...,0.007576,0.006639,-0.105099,0.082139,0.258052,0.175949,0.073437,0.155727,Analysis,1
3,0.059827,-0.062374,0.058713,0.079709,0.041607,0.097148,0.185068,0.175076,0.037400,-0.011215,...,0.007576,0.006639,-0.105099,0.082139,0.258052,0.175949,0.073437,0.155727,Analysis,1
4,0.059827,-0.062374,0.058713,0.079709,0.041607,0.097148,0.185068,0.175076,0.037400,-0.011215,...,0.007576,0.006639,-0.105099,0.082139,0.258052,0.175949,0.073437,0.155727,Analysis,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448580,-0.035866,-0.133072,0.113057,-0.015819,0.091003,0.212770,0.170348,0.118591,0.041808,0.021649,...,0.056514,-0.036542,-0.031596,0.124603,0.245927,0.229201,0.112923,0.176976,Benign,0
448581,-0.035866,-0.133072,0.113057,-0.015819,0.091003,0.212770,0.170348,0.118591,0.041808,0.021649,...,0.056514,-0.036542,-0.031596,0.124603,0.245927,0.229201,0.112923,0.176976,Benign,0
448582,-0.035866,-0.133072,0.113057,-0.015819,0.091003,0.212770,0.170348,0.118591,0.041808,0.021649,...,0.056514,-0.036542,-0.031596,0.124603,0.245927,0.229201,0.112923,0.176976,Benign,0
448583,-0.035866,-0.133072,0.113057,-0.015819,0.091003,0.212770,0.170348,0.118591,0.041808,0.021649,...,0.056514,-0.036542,-0.031596,0.124603,0.245927,0.229201,0.112923,0.176976,Benign,0


# Embeddings CBLOF  Embeddings

In [39]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [40]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [41]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.097376,-0.059250,0.106376,0.118931,-0.039355,0.023157,0.234994,0.178349,0.128302,0.030925,...,0.384168,0.121901,0.022039,0.061053,-0.166547,0.071712,0.211216,0.126841,0.045681,0.232362
1,0.097376,-0.059250,0.106376,0.118931,-0.039355,0.023157,0.234994,0.178349,0.128302,0.030925,...,0.384168,0.121901,0.022039,0.061053,-0.166547,0.071712,0.211216,0.126841,0.045681,0.232362
2,0.097376,-0.059250,0.106376,0.118931,-0.039355,0.023157,0.234994,0.178349,0.128302,0.030925,...,0.384168,0.121901,0.022039,0.061053,-0.166547,0.071712,0.211216,0.126841,0.045681,0.232362
3,0.097376,-0.059250,0.106376,0.118931,-0.039355,0.023157,0.234994,0.178349,0.128302,0.030925,...,0.384168,0.121901,0.022039,0.061053,-0.166547,0.071712,0.211216,0.126841,0.045681,0.232362
4,0.097376,-0.059250,0.106376,0.118931,-0.039355,0.023157,0.234994,0.178349,0.128302,0.030925,...,0.384168,0.121901,0.022039,0.061053,-0.166547,0.071712,0.211216,0.126841,0.045681,0.232362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478049,0.001657,-0.071243,0.096393,0.018465,0.063533,0.099651,0.213297,0.103149,0.111735,0.042132,...,0.391929,0.065964,0.028198,0.016128,-0.086237,0.113210,0.264520,0.203477,0.077756,0.207359
478050,0.001657,-0.071243,0.096393,0.018465,0.063533,0.099651,0.213297,0.103149,0.111735,0.042132,...,0.391929,0.065964,0.028198,0.016128,-0.086237,0.113210,0.264520,0.203477,0.077756,0.207359
478051,0.001657,-0.071243,0.096393,0.018465,0.063533,0.099651,0.213297,0.103149,0.111735,0.042132,...,0.391929,0.065964,0.028198,0.016128,-0.086237,0.113210,0.264520,0.203477,0.077756,0.207359
478052,0.074274,-0.031197,0.093040,0.098063,0.011389,0.064751,0.254288,0.128092,0.144965,0.041076,...,0.387072,0.065996,0.071466,0.032453,-0.118112,0.081318,0.279250,0.176921,0.104779,0.187381


In [42]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [43]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
28228,1.719763e-06,4.298851e-06,0.028485,0.000341,0.045794,0.000341,5.420943e-07,5.797048e-07,2.172674e-05,0.024089,...,0.558214,0.558214,6.541834e-06,6.541834e-06,1.597788e-06,1.597954e-06,0.0,1.540128e-06,Analysis,1
856405,1.379976e-06,3.449494e-06,0.022255,0.000273,0.035816,0.000219,4.349885e-07,4.651680e-07,1.743402e-05,0.011921,...,0.447924,0.447924,9.316029e-06,9.247537e-06,1.282101e-06,1.282234e-06,0.0,1.235833e-06,Analysis,1
465045,1.987601e-06,4.968358e-06,0.031897,0.000394,0.043554,0.000315,6.265204e-07,6.699884e-07,2.511048e-05,0.042687,...,0.645151,0.645151,4.500492e-06,4.642276e-06,1.846629e-06,1.846821e-06,0.0,1.779989e-06,Analysis,1
959848,1.865023e-06,4.661954e-06,0.029782,0.000370,0.047666,0.000370,5.878823e-07,6.286695e-07,2.356189e-05,0.032554,...,0.605364,0.605364,1.305940e-05,1.283819e-05,1.732745e-06,1.732925e-06,0.0,1.670215e-06,Analysis,1
1381572,1.770291e-06,4.425153e-06,0.028129,0.000351,0.038160,0.000281,5.580212e-07,5.967367e-07,2.236508e-05,0.022938,...,0.574615,0.574615,1.632465e-05,1.632465e-05,1.644732e-06,1.644903e-06,0.0,1.585378e-06,Analysis,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442625,1.242158e-06,3.104993e-06,0.031895,0.000246,0.006300,0.000148,2.454900e-05,9.606428e-06,1.569289e-05,0.007777,...,0.403190,0.403190,4.263192e-06,4.422153e-06,1.154057e-06,1.154177e-06,0.0,1.112411e-06,Worms,1
1417188,1.844046e-06,4.609518e-06,0.047350,0.000365,0.009353,0.000219,3.644422e-05,1.426122e-05,2.329687e-05,0.026707,...,0.598555,0.598555,6.328921e-06,6.564907e-06,1.713256e-06,1.713434e-06,0.0,1.651429e-06,Worms,1
892190,7.800834e-07,1.949956e-06,0.020030,0.000155,0.003957,0.000093,1.541693e-05,6.032898e-06,9.855233e-06,0.002689,...,0.253206,0.253206,2.677312e-06,2.777141e-06,7.247555e-07,7.248308e-07,0.0,6.986009e-07,Worms,1
763320,2.017698e-06,5.043591e-06,0.051728,0.000400,0.013432,0.000320,3.987613e-05,1.560419e-05,2.549071e-05,0.047051,...,0.654920,0.654920,1.319737e-05,1.369353e-05,1.874591e-06,1.874786e-06,0.0,1.806942e-06,Worms,1


In [44]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [45]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [46]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:46<00:00,  2.95s/it]

{'n_estimators': 2, 'con': 0.001}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [47]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:54<00:00,  3.17s/it]

{'n_estimators': 2, 'con': 0.05}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [48]:
###  CBLOF RAW

In [49]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:07,  4.68it/s]

2


  8%|▊         | 3/36 [00:01<00:16,  1.98it/s]

2


 14%|█▍        | 5/36 [00:01<00:09,  3.13it/s]

...

3
3


 33%|███▎      | 12/36 [00:03<00:05,  4.72it/s]

3


100%|██████████| 36/36 [00:19<00:00,  1.84it/s]

{'n_estimators': 7, 'con': 0.04}
0.6501820674302449
              precision    recall  f1-score   support

           0     0.9791    0.9377    0.9580    229522
           1     0.2559    0.5173    0.3424      9505

    accuracy                         0.9210    239027
   macro avg     0.6175    0.7275    0.6502    239027
weighted avg     0.9504    0.9210    0.9335    239027



In [50]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:00<00:13,  2.61it/s]

2


 11%|█         | 4/36 [00:01<00:12,  2.58it/s]

2


 17%|█▋        | 6/36 [00:02<00:11,  2.55it/s]

...

3


 33%|███▎      | 12/36 [00:03<00:05,  4.30it/s]

3


100%|██████████| 36/36 [00:20<00:00,  1.77it/s]

benign only
{'n_estimators': 7}
0.7153040747023789
              precision    recall  f1-score   support

           0     0.9785    0.9733    0.9759    229522
           1     0.4287    0.4841    0.4547      9505

    accuracy                         0.9538    239027
   macro avg     0.7036    0.7287    0.7153    239027
weighted avg     0.9567    0.9538    0.9552    239027



In [51]:
# HBOS  Embeddings

In [52]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [53]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [04:29<00:00,  7.50s/it]

{'n_estimators': 5, 'con': 0.001}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [54]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [04:29<00:00,  7.49s/it]

{'n_estimators': 5, 'con': 0.05}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [55]:
##  HBOS  RAw

In [56]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:20<00:00,  1.79it/s]

{'n_estimators': 5, 'con': 0.2}
0.5270372524403804
              precision    recall  f1-score   support

           0     0.9628    0.9507    0.9567    229522
           1     0.0861    0.1120    0.0974      9505

    accuracy                         0.9174    239027
   macro avg     0.5244    0.5314    0.5270    239027
weighted avg     0.9279    0.9174    0.9225    239027



In [57]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:18<00:00,  1.93it/s]

benign only
{'n_estimators': 5}
0.5306929687337104
              precision    recall  f1-score   support

           0     0.9629    0.9560    0.9594    229522
           1     0.0944    0.1108    0.1019      9505

    accuracy                         0.9224    239027
   macro avg     0.5287    0.5334    0.5307    239027
weighted avg     0.9284    0.9224    0.9253    239027



In [58]:
##  PCA  Emb

In [59]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:41<00:00,  2.83s/it]

{'n_estimators': 5, 'con': 0.001}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [60]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:42<00:00,  2.83s/it]

{'n_estimators': 5, 'con': 0.04}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [61]:
##  PCA  RAw

In [62]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:16<00:00,  2.14it/s]

{'n_estimators': 10, 'con': 0.01}
0.5119911291133447
              precision    recall  f1-score   support

           0     0.9620    0.9248    0.9431    229522
           1     0.0614    0.1187    0.0809      9505

    accuracy                         0.8928    239027
   macro avg     0.5117    0.5218    0.5120    239027
weighted avg     0.9262    0.8928    0.9088    239027



In [63]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:16<00:00,  2.15it/s]

benign only
{'n_estimators': 25}
0.51558002781282
              precision    recall  f1-score   support

           0     0.9622    0.9316    0.9467    229522
           1     0.0661    0.1169    0.0845      9505

    accuracy                         0.8992    239027
   macro avg     0.5142    0.5243    0.5156    239027
weighted avg     0.9266    0.8992    0.9124    239027



In [64]:
##  IF  Emb

In [65]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:09<00:00,  2.90s/it]

{'n_estimators': 20, 'con': 0.001}
0.03824457212753287
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    459044
           1     0.0398    1.0000    0.0765     19010

    accuracy                         0.0398    478054
   macro avg     0.0199    0.5000    0.0382    478054
weighted avg     0.0016    0.0398    0.0030    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [66]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:16<00:00,  3.18s/it]

{'n_estimators': 20, 'con': 0.01}
0.489856984008076
              precision    recall  f1-score   support

           0     0.9602    1.0000    0.9797    459044
           1     0.0000    0.0000    0.0000     19010

    accuracy                         0.9602    478054
   macro avg     0.4801    0.5000    0.4899    478054
weighted avg     0.9221    0.9602    0.9408    478054




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [67]:
##  IF  Raw

In [68]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:23<00:00,  1.04it/s]

{'n_estimators': 20, 'con': 0.2}
0.5442573970752405
              precision    recall  f1-score   support

           0     0.9625    0.9981    0.9800    229522
           1     0.5711    0.0600    0.1085      9505

    accuracy                         0.9608    239027
   macro avg     0.7668    0.5291    0.5443    239027
weighted avg     0.9469    0.9608    0.9453    239027



In [69]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:23<00:00,  1.03it/s]

benign only
{'n_estimators': 100}
0.5080318453132209
              precision    recall  f1-score   support

           0     0.9608    0.9802    0.9704    229522
           1     0.0674    0.0345    0.0456      9505

    accuracy                         0.9426    239027
   macro avg     0.5141    0.5074    0.5080    239027
weighted avg     0.9253    0.9426    0.9336    239027

